# Lecture 12 - Dimensionality Reduction: Principal Component Analysis

## Objectives

+ Objective 1
+ Objective 2

### Karhunen-Loeve Expansion.

Consider a Gaussian random field over a set $\Omega \subset \R^d$:
\begin{equation}
f(\cdot) | m(\cdot), k(\cdot, \cdot) \sim \GP\left(f(\cdot) | m(\cdot), k(\cdot, \cdot) \right),
\end{equation}
where 
$m:\Omega\rightarrow R$ is the mean function and 
$k:\Omega\times\Omega\rightarrow\R$ is the covariance function.
The *Karhunen-Loève expansion* (KLE) of $f(\cdot)$ allows us to write it as:
\begin{equation}
f(\bx) = m(\bx) + \sum_{i=1}^{\infty}\xi_i \sqrt{\lambda_i} \phi_i(\bx),
\end{equation}
where the random variables
\begin{equation}
\xi \sim \calN(\xi|0, 1)
\end{equation}
are independent, and $\lambda_i$ and $\phi_i(\bx)$ are the eigenvalues and
eigenvectors, respectively, of the covariance function, i.e.,
\begin{equation}
\int_{\Omega} k(\bx, \bx') \phi_i(\bx')d\bx' = \lambda_i \phi_i(\bx').
\end{equation}
Since $k(\bx, \bx')$ is actually positive definite, the eigenvalues are all positive
and the eigenfunctions are orthogonal:
$$
\int_{\Omega} \phi_i(\bx)\phi_j(\bx')d\bx = \delta_{ij}.
$$

### Truncated KLE
Usually, we truncate the KLE to a finite order $M$, i.e., we write
\begin{equation}
f(\bx) \approx f_M(\bx) = m(\bx) + \sum_{i=1}^M \xi_i \sqrt{\lambda_i}\phi_i(\bx).
\end{equation}
But how do we pick $M$ in practice?

In order to answer this question, notice that the variance of the field at the point $\bx$ is given by:
$$
\mathbb{V}[f(\bx)] = \sum_{i=1}^{\infty}\lambda_i\phi_i^2(\bx)
$$
The *energy* of the field, $\mathcal{E}[f(\cdot)]$ is defined to be:
\begin{equation}
\mathcal{E}[f(\cdot)] := \int_{\Omega}\mathbb{V}[f(\bx)]d\bx = \sum_{i=1}^\infty \lambda_i,
\end{equation}
where we have used the orthonormality of the $\phi(\bx)$'s.
The energy of the field is a measure of the total variance of the field.
The idea is to select $M$ so that the energy of the truncated field $f_M(\bx)$ is as
captures a percentage $\alpha$ of the energy of the original field.
That is, we pick $M$ so that
$$
\mathcal{E}[f_M(\cdot)] = \alpha\mathcal{E}[f(\cdot)],
$$
or
$$
\sum_{i=1}^M\lambda_i = \alpha \sum_{i=1}^\infty \lambda_i.
$$
Typically, $\alpha = 0.95$.

### Why is this useful?
The KLE allows us to reduce the dimensionality of random fields.
This is extremely useful in uncertainty propagation and model calibration tasks.
For example, in uncertainty propagation, by employing the KLE one has to deal with
a finite set of Gaussian random variables $\xi_i$ instead of an infinitely dimensional
Gaussian random field.

### Numerical computation of KLE
Solving the integral equation for the eigenvalues and eigenvectors of $k(\bx,\bx')$ is actually very difficult.
Instead, one uses a quadrature rule on the space of $\bx$, $\{w_j, \bx_j\}_{j=1}^N$,
and approximates the left hand side of the integral equation as follows:
$$
\sum_{j=1}^N w_j k(\bx, \bx_j)\phi_i(\bx_j) = \lambda_i \phi_i(\bx).
$$
From this, we see that we can express the eigenfunctions as:
$$
\phi_i(\bx) = \lambda_i^{-1}\sum_{j=1}^Nw_jk(\bx, \bx_j)\phi_i(\bx_j),
$$
if we new the eigenvalues, as well as their values of the quadrature points.

In order to approximate $\lambda_i$ and $\phi_i(\bx_j)$, we consider the following finite dimensional eigenvalue
problem:
$$
\sum_{j=1}^N w_j k(\bx_i, \bx_j) v_j = \mu v_i,
$$
or in matrix form:
$$
\bK \mbox{diag}(w) \bv = \mu \bv,
$$
where
$$
K_{ij} = k(\bx_i, \bx_j).
$$
Assuming that by solving it, we obtain the eigenvalues $\mu_i$ and the eigenvectors
$\bv_i$ for $i=1,\dots,N$, we estimate that:
$$
\lambda_i \approx \mu_i,\;\mbox{for}\;$i=1,\dots,N$,
$$
$$
\lambda_i \approx 0,\;\mbox{for}\;$i>N$,
$$
and
$$
\phi_i(\bx_j) = v_{ij},
$$
for $i,j=1,\dots,N$.
Returning back to the full eigenfunctions, we see that they can be expressed as:
$$
\phi_i(\bx) = \mu^{-1}_i\sum_{j=1}^{N}v_{ij} w_j k(\bx_j, \bx),
$$
for $i=1,\dots,N$.

The most common choice of quadrature points is to pick $w_i = \frac{1}{N}$ and
$\bx_i$ to be either random or on a grid. We will try a few.